# Spectral Gradient

In this exercise, we will measure the spectral gradient of a comet nucleus.  The data set is comet 322P/SOHO as measured by Knight et al. (2016, ApJ 823, L6).

## Data

Knight et al. observed 322P with the FORS2 instrument on VLT, with the *g_High*, *R_special*, and *i_Bessel* filters.  The data were calibrated to the SDSS photometric system (Smith et al. 2002, BAAS 34, 1272), resulting in *g'-r'* and *r'-i'* colors.

## Top-level imports

We need astropy's ``units``, and the `SpectralGradient` class from sbpy.  The SDSS filter bandpasses are distributed with sbpy, and retrievable via `bandpass`.  We will also use sbpy's solar spectrum.

In [1]:
import astropy.units as u
from sbpy.spectroscopy import SpectralGradient
from sbpy.photometry import bandpass
from sbpy.calib import Sun

## Define input data

SDSS uses the AB-magnitude system:

In [2]:
g = bandpass('SDSS g')
r = bandpass('SDSS r')
i = bandpass('SDSS i')

gmr = 0.52 * u.mag  # ±0.04
rmi = 0.03 * u.mag  # ±0.06

## Ratio with the Sun

`SpectralGradient` requires observations in units of reflectance, i.e., the spectrum of the Sun must be removed.  Calculate the color of the Sun in the same units as the SDSS photometric system (AB magnitudes).

In [3]:
sun = Sun.from_default()

lambda_eff, color = sun.color_index((g, r), u.ABmag)
print("Sun(g'-r') = {:.2f}".format(color))
gmr = gmr - color

lambda_eff, color = sun.color_index((r, i), u.ABmag)
print("Sun(r'-i') = {:.2f}".format(color))
rmi = rmi - color

Sun(g'-r') = 0.48 mag
Sun(r'-i') = 0.11 mag


## Calculate spectral gradient

In [4]:
for label, filters, color in (("g'-r'", (g, r), gmr), ("r'-i'", (r, i), rmi)):
    S = SpectralGradient.from_color(filters, color)
    print("{} = {:.2f} → S({}) = {:.1f}".format(label, color, label, S))

g'-r' = 0.04 mag → S(g'-r') = 2.6 % / 100 nm
r'-i' = -0.08 mag → S(r'-i') = -5.4 % / 100 nm
